In [1]:
# file related
import os
from os.path import join
import datetime

# machine learning
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

# utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import Audio as play_audio
import scipy as sp
import taunet_utils
import json

# RTNeural special import
import sys
sys.path.append("../RTNeural/python/")
from model_utils import save_model

print(f"TF version {tf.__version__}")

TF version 2.12.0


In [2]:
# input_data, output_data = taunet_utils.create_dataset(join("dataset", "audio"), 
#                                                       join("dataset", "human_input", "AITD_Dataset_Kristof_beta_1.csv"),
#                                                       join("..", "src", "af", "AFInC.dll"),
#                                                       join("dataset", "saved", "AITD_Dataset_Kristof_beta_1.json"))

with open(join("dataset", "saved", "AITD_Dataset_Kristof_beta_1in.json"), "r") as json_file:
    input_data = json.load(json_file)
with open(join("dataset", "saved", "AITD_Dataset_Kristof_beta_1out.json"), "r") as json_file:
    output_data = json.load(json_file)

for d in input_data:
    print(f"{d}: {input_data[d]} -> {output_data[d]}")

1: [123.9669, 0.012, 0.2251, 15648, 0.9176, 82, 10, 5, 1, 47.5291, 0.6738, 0.8758, 550.0, 350.0] -> [360.5123, 1144.7427]
2: [153.0612, 0.0322, 0.1695, 10716, 0.8286, 137, 7, 3, 2, 94.4725, 0.7384, 0.2437, 550.0, 350.0] -> [499.9752, 374.6085]
3: [171.1027, 0.0236, 0.3093, 6128, 0.807, 72, 7, 5, 1, 79.7098, 0.2224, 0.9991, 550.0, 350.0] -> [22.6204, 1932.1606]
4: [181.4516, 0.0294, 0.312, 7848, 0.6281, 82, 9, 5, 1, 53.2918, 0.6711, 0.8347, 550.0, 350.0] -> [149.4464, 1335.9675]
5: [108.6957, 0.012, 0.2543, 14056, 0.788, 110, 6, 3, 2, 100.8992, 0.6149, 0.8181, 550.0, 350.0] -> [351.3529, 1696.3634]
6: [173.7452, 0.0087, 0.1529, 25353, 0.6351, 3, 3, 4, 3, 188.9978, 0.2, 0.9179, 550.0, 350.0] -> [91.6004, 2420.8156]
7: [186.722, 0.0172, 0.3052, 18645, 0.8797, 67, 8, 3, 2, 88.6245, 0.6255, 0.4053, 550.0, 350.0] -> [499.9876, 221.555]
8: [199.115, 0.0269, 0.1893, 3541, 1.1604, 97, 6, 3, 1, 25.3582, 0.6357, 0.3034, 550.0, 350.0] -> [470.3991, 217.7039]
9: [119.6809, 0.0604, 0.4037, 3407, 1.3

In [3]:
def create_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    # model.add(layers.LSTM(64, return_sequences=True))   # short term memory, useful if input data is related accross vectors
    model.add(Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.001))) # no activation (linear): continuous mapping of outputs (this is not a classification task!)
    return model

input_shape = (14,)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                960       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 4)                 36        
                                                                 
 dense_6 (Dense)             (None, 2)                 1

In [4]:
combined_data = [(input_data[key], output_data[key]) for key in input_data.keys()]

train_data, test_data = train_test_split(combined_data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.05, random_state=42)

log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

class LogRMSECallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super(LogRMSECallback, self).__init__()
        self.log_dir = log_dir

    def on_epoch_end(self, epoch, logs=None):
        mse = logs.get('val_loss')
        rmse = np.sqrt(mse)
        with tf.summary.create_file_writer(self.log_dir).as_default():
            tf.summary.scalar('val_rmse', rmse, step=epoch)

%load_ext tensorboard
%tensorboard --logdir logs/ --port 6006

history = model.fit(
    x=np.array([item[0] for item in train_data]),
    y=np.array([item[1] for item in train_data]),
    validation_data=(
        np.array([item[0] for item in val_data]),
        np.array([item[1] for item in val_data])
    ),
    epochs=80,
    # callbacks=[tensorboard_callback, early_stopping_callback],
    callbacks=[tensorboard_callback, early_stopping_callback, LogRMSECallback(join(log_dir, "rmse"))],
    # batch_size=16
)

Reusing TensorBoard on port 6006 (pid 11700), started 3 days, 21:08:59 ago. (Use '!kill 11700' to kill it.)

Epoch 1/80
2/2 [==============================] - 3s 602ms/step - loss: 886319.1875 - accuracy: 0.8800 - val_loss: 680546.8125 - val_accuracy: 0.6667
Epoch 2/80
2/2 [==============================] - 0s 163ms/step - loss: 836673.6250 - accuracy: 0.8800 - val_loss: 672512.5000 - val_accuracy: 0.6667
Epoch 3/80
2/2 [==============================] - 0s 159ms/step - loss: 809420.1875 - accuracy: 0.8800 - val_loss: 663762.3125 - val_accuracy: 0.6667
Epoch 4/80
2/2 [==============================] - 0s 157ms/step - loss: 779153.3750 - accuracy: 0.8800 - val_loss: 656800.6250 - val_accuracy: 0.6667
Epoch 5/80
2/2 [==============================] - 0s 168ms/step - loss: 751862.6250 - accuracy: 0.8800 - val_loss: 644985.6875 - val_accuracy: 0.6667
Epoch 6/80
2/2 [==============================] - 0s 156ms/step - loss: 717817.2500 - accuracy: 0.8800 - val_loss: 633204.6875 - val_accuracy: 0.6667
Epoch 7/80
2/2 [==============================] - 0s 160ms/step - loss: 682483.2500 - accuracy: 0.88

In [5]:
train_metrics = model.evaluate(
    np.array([item[0] for item in train_data]),
    np.array([item[1] for item in train_data]),
    verbose=0
)
val_metrics = model.evaluate(
    np.array([item[0] for item in val_data]),
    np.array([item[1] for item in val_data]),
    verbose=0
)
test_metrics = model.evaluate(
    np.array([item[0] for item in test_data]),
    np.array([item[1] for item in test_data]),
    verbose=0
)

for item in test_data:
    inp = np.array([item[0]])  # Reshape input data into a batch
    prediction = model.predict(inp)
    print("Input:", item[0])
    print("Expected Output:", item[1])
    print("Predicted Output:", prediction)


print({f"Train {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, train_metrics)})
print({f"Validation {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, val_metrics)})
print({f"Test {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, test_metrics)})
print("RMSE for test set:", np.sqrt(test_metrics[0]))

# save model as .h5, .tflite and .json
model.save(join(log_dir, "taunet.pb"))
tflite_model = tf.lite.TFLiteConverter.from_saved_model(join(log_dir, "taunet.pb")).convert()
with open(join(log_dir, "taunet.tflite"), 'wb') as f:
  f.write(tflite_model)
save_model(model, join(log_dir, "taunet.json"))

1/1 [==============================] - 0s 135ms/step
Input: [123.9669, 0.012, 0.2251, 15648, 0.9176, 82, 10, 5, 1, 47.5291, 0.6738, 0.8758, 550.0, 350.0]
Expected Output: [360.5123, 1144.7427]
Predicted Output: [[ -46.935417 1496.0387  ]]
1/1 [==============================] - 0s 23ms/step
Input: [173.7452, 0.0087, 0.1529, 25353, 0.6351, 3, 3, 4, 3, 188.9978, 0.2, 0.9179, 550.0, 350.0]
Expected Output: [91.6004, 2420.8156]
Predicted Output: [[ -74.492905 2371.4602  ]]
1/1 [==============================] - 0s 23ms/step
Input: [127.1186, 0.0326, 0.2023, 10029, 0.6916, 58, 5, 5, 2, 8.9219, 0.79, 0.7839, 550.0, 350.0]
Expected Output: [435.8906, 1932.2294]
Predicted Output: [[-31.22067 996.8269 ]]
1/1 [==============================] - 0s 23ms/step
Input: [152.5424, 0.0422, 0.245, 2114, 1.2588, 24, 6, 6, 1, 18.9737, 0.6313, 0.2773, 550.0, 350.0]
Expected Output: [281.9786, 461.7476]
Predicted Output: [[ -9.0678835 293.09726  ]]
1/1 [==============================] - 0s 27ms/step
Input: [1

INFO:tensorflow:Assets written to: logs/20240429-125746\taunet.pb\assets


INFO:tensorflow:Assets written to: logs/20240429-125746\taunet.pb\assets
